In [0]:
!pip install face-recognition

In [0]:
from google.colab import drive
drive.mount("drive/")

In [0]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import face_recognition
from PIL import Image

## BLURRING EFFECTS ##

In [0]:
motion_blur_kernel_range = (8,12)
average_blur_kernel_range = (3,9)
gaussian_blur_kernel_range = (3,10)

In [0]:
Blurring_Kernels = ["motion","gaussian","average"]

In [0]:
def motion_blur(img):

  kernel_size = np.random.randint(motion_blur_kernel_range[0],motion_blur_kernel_range[1])
  kernel = np.zeros((kernel_size, kernel_size))

  types = ["vertical","horizontal","main_diagonal","anti_diagonal"]

  choice = np.random.choice(types)

  if choice=="vertical":
    kernel[:,int((kernel_size-1)/2)] = np.ones(kernel_size)/kernel_size
  
  elif choice=="horizontal":
    kernel[int((kernel_size-1)/2),:] = np.ones(kernel_size)/kernel_size
  
  elif choice=="main_diagonal":
    
    for i in range(kernel_size):
      kernel[i][i] = 1/kernel_size

  elif choice=="anti_diagonal":

    for i in range(kernel_size):
      kernel[i][kernel_size-i-1] = 1/kernel_size

  return cv2.filter2D(face,-1,kernel)

In [0]:
def get_blurred_face(face):
  
  choice = np.random.choice(Blurring_Kernels)

  if choice=="motion":
  
    random_blurred_face  = motion_blur(face)

  elif choice=="gaussian":

    kernel_size = np.random.randint(gaussian_blur_kernel_range[0],gaussian_blur_kernel_range[1])

    if kernel_size%2==0:
      kernel_size-=1
    
    random_blurred_face = cv2.GaussianBlur(face,(kernel_size,kernel_size),0)

  elif choice=="average":
  
    kernel_size = np.random.randint(average_blur_kernel_range[0],average_blur_kernel_range[1])
    random_blurred_face = cv2.blur(face,(kernel_size,kernel_size))

  return random_blurred_face

## MASK FACE ##

In [0]:
def get_distance_from_point_to_line(point, line_point1, line_point2):

        (p,q) = point
        (x1,y1) = line_point1
        (x2,y2) = line_point2

        A = (y2-y1)
        B = (x1-x2)
        
        distance = np.abs( A*p + B*q  - A*x1 - B*y1 ) / np.sqrt( A**2 + B**2 )

        return int(distance)

In [0]:
def get_point_rotation(point,origin,angle):

        (p,q) = point
        (x,y) = origin

        rotated_p = x + np.cos(angle)*(p-x) - np.sin(angle)*(q-y)
        rotated_q = y + np.sin(angle)*(p-x) + np.cos(angle)*(q-y)

        return [int(rotated_p),int(rotated_q)]

In [0]:
def Mask_Faces (image_path,mask_image_path):

  masked_faces_image = Image.open(image_path)

  image = face_recognition.load_image_file(image_path)
  mask_img = Image.open(mask_image_path)

  face_landmarks = face_recognition.face_landmarks(image)

  if len(face_landmarks)==0:
    print("No Faces found in "+image_path.split('/')[-1]+" !")

  for face in face_landmarks:

    if 'nose_bridge' not in face or 'chin' not in face :
      continue

    nose_bridge = face['nose_bridge']
    nose_point = np.array(nose_bridge[len(nose_bridge) * 1 // 4])

    chin = face['chin']
    chin_len = len(chin)

    chin_bottom_point = np.array(chin[chin_len // 2])       
    chin_left_point = np.array(chin[chin_len // 8])        
    chin_right_point = np.array(chin[chin_len * 7 // 8])

    width = mask_img.width
    height = mask_img.height        
    width_ratio = 1.15       
    new_mask_height = int(np.linalg.norm(nose_point - chin_bottom_point))

    mask_left_img = mask_img.crop((0, 0, width // 2, height))        
    mask_left_width = get_distance_from_point_to_line(chin_left_point, nose_point, chin_bottom_point)        
    mask_left_width = int(mask_left_width * width_ratio)        
    mask_left_img = mask_left_img.resize((mask_left_width, new_mask_height))
        
    mask_right_img = mask_img.crop((width // 2, 0, width, height))        
    mask_right_width = get_distance_from_point_to_line(chin_right_point, nose_point, chin_bottom_point)        
    mask_right_width = int(mask_right_width * width_ratio)        
    mask_right_img = mask_right_img.resize((mask_right_width, new_mask_height))


    new_mask_size = (mask_left_img.width + mask_right_img.width, new_mask_height)        
    new_mask_img = Image.new('RGBA', new_mask_size)        
    new_mask_img.paste(mask_left_img, (0, 0), mask_left_img)        
    new_mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

    angle_radian = np.arctan2(chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
    rotation_angle_radian = (np.pi/2) - angle_radian
    rotation_angle_degree = (rotation_angle_radian*180)/np.pi

    rotation_center = (mask_left_width,new_mask_height//2)

    rotated_mask_img = new_mask_img.rotate(rotation_angle_degree,expand=True,center = rotation_center)